# Forecasting & Inventory Alert System

An end-to-end ingredient demand forecasting & inventory alert system for a café, designed from raw transactional data to procurement signals

Creator: [Andrew Virya Victorio](https://avv.framer.website/) <br>
Email: [andrew.avv03@gmail.com](mailto:andrew.avv03@gmail.com) <br>
Repository: [github.com/AlphaByte-RedTeam/forecasting-inventory-alert-system](https://github.com/AlphaByte-RedTeam/forecasting-inventory-alert-system) <br>

Created date: Dec 17th, 2025

## Load the dataset

In [10]:
import pandas as pd
import os

current_dir = os.getcwd()
dataset_dir = f'{current_dir}/datasets'

df_inventory = pd.read_csv(f'{dataset_dir}/coffee_shop_inventory_2y.csv')
df_sales = pd.read_csv(f'{dataset_dir}/coffee_shop_sales_detailed_2y.csv')

In [11]:
df_inventory.head()

,date,ingredient_name,ingredient_unit,opening_stock,used_today,closing_stock,reorder_point,lead_time_days,reorder_flag
0,2023-01-01,coffee,g,50000.0,5004.0,44996.0,15000,7,False
1,2023-01-01,milk,ml,200000.0,29420.0,170580.0,60000,3,False
2,2023-01-01,chocolate,g,30000.0,850.0,29150.0,8000,5,False
3,2023-01-01,sugar,g,40000.0,3094.5,36905.5,10000,4,False
4,2023-01-02,coffee,g,44996.0,2610.0,42386.0,15000,7,False


In [12]:
df_sales.head()

,date,product,temperature,sugar_level,ice_level,coffee_used,coffee_unit,milk_used,milk_unit,chocolate_used,chocolate_unit,sugar_used,sugar_unit,water_used,water_unit,is_weekend,is_promo
0,2023-01-01,Espresso,hot,extra,none,18,g,0,ml,0,g,12.0,g,30.0,ml,True,True
1,2023-01-01,Espresso,hot,normal,none,18,g,0,ml,0,g,10.0,g,30.0,ml,True,True
2,2023-01-01,Espresso,hot,normal,none,18,g,0,ml,0,g,10.0,g,30.0,ml,True,True
3,2023-01-01,Espresso,ice,normal,normal,18,g,0,ml,0,g,10.0,g,30.0,ml,True,True
4,2023-01-01,Espresso,hot,less,none,18,g,0,ml,0,g,7.5,g,30.0,ml,True,True
